# 0.reRank_reOrder_multi_query 

##   Ollama-ko-0710:latest 와 gpt-4.1-mini 비교

### multi query 생성시 gpt-4.1-mini 사용

### ollama-ko-0710:latest 답변

- 질문: ['삼성전자의 하반기 실적 예측은 어떻게 되나요?  ', '올해 하반기 삼성전자의 재무 성과는 어떤가요?  ', '삼성전자의 2023년 하반기 실적에 대한 전망은 무엇인가요?']
- 답변: 삼성전자의 하반기 실적 전망은 긍정적인 것으로 나타나고 있습니다. 미래에셋증권과 같은 많은 증권사들은 삼성전자가 하반기에 나쁜 결과를 기록할 것이라고 전망하고 있습니다. 실제로 NH투자증권도 최근 삼성전자 목표가를 9만원 이상으로 상향 조정하면서 긍정적인 시각을 나타냈습니다. 특히, 파운드리 부문에서 테슬라와 애플 등 큰 고객사를 확보한 점과 1C(10나노급 6세대) 수율 개선과 함께 하반기 엔비디아 고대역폭 메모리(HBM) 공급 가능성에 대한 기대감이 실적 향상에 긍정적인 영향을 미칠 것으로 예상됩니다. 또한, 삼성전자가 높은 수율을 유지하면서 매출이 증가할 가능성도 높습니다.

반면, 한국투자증권은 하반기 실적 전망을 주의보로 내린 바 있습니다. 이는 글로벌 경제 불확실성과 IT 기기 수요가 상대적으로 안정적인 단계에서 현재를 보는 것에 기반한 평가입니다. 그러나, 삼성전자의 강력한 기술 경쟁력과 글로벌 시장에서의 독자성 덕분에 중장기적인 성장 가능성이 크다고 볼 수 있습니다.

결론적으로, 삼성전자의 하반기 실적 전망은 긍정적인 측면이 더 두드러지지만, 단기적인 경제 불확실성의 영향을 반영하여 주의가 필요하다는 점에서 복합적인 평가를 내릴 수 있습니다.

### gpt-4.1-mini 

- 질문:  ['삼성전자의 2023년 하반기 실적 예측은 어떻게 되나요?  ', '올해 하반기 삼성전자의 재무 성과에 대한 전망은 무엇인가요?  ', '삼성전자의 하반기 실적에 대한 분석이나 예측이 있나요?']
- 답변: '삼성전자의 하반기 실적 전망은 긍정적입니다. NH투자증권 류영호 연구원은 파운드리 부문에서 테슬라와 애플 등 주요 고객사를 확보하고, 10나노급 6세대 공정 수율 개선, 그리고 하반기 엔비디아 고대역폭 메모리(HBM) 공급 기대감이 주가에 긍정적인 영향을 미칠 것이라고 분석했습니다. 또한, 3분기 영업이익은 9조 원으로 전년 동기 대비 1.9% 감소하나 직전 분기 대비 91.9% 증가할 것으로 전망됩니다. 미래에셋증권도 올해와 내년 영업이익 전망치를 각각 2.9%, 5.3% 상향 조정하며 목표 주가를 9만6000원으로 올렸습니다. 이는 내년 메모리 반도체 공급 부족과 차세대 고대역폭 메모리 생산 집중에 따른 가격 상승이 기대되기 때문입니다. 따라서 삼성전자의 하반기 실적은 개선될 가능성이 높다고 볼 수 있습니다.'

### 결과

- 질문 : 삼성전자 하반기 실적 전망은?
- 데이터 : 네이버 뉴스 mcp 서버 이용하여 얻은 "삼성전자 주가" query 로 얻은 100개의 데이터중 네이버 뉴스에 등록된 데이터 36개 크롤링하여 본문만 저장한 데이터 

### retriver 생성
```
retriever = db.as_retriever(
    search_type="mmr", search_kwargs={"k": 10, "lambda_mult": 0.25, "fetch_k": 20}
)
```

### multi query retriever
```
def multi_query_retriever(retriever):
    # 다중 질의어 생성
    llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")
    multiquery_retriever = MultiQueryRetriever.from_llm(  # MultiQueryRetriever를 언어 모델을 사용하여 초기화합니다.
    # 벡터 데이터베이스의 retriever와 언어 모델을 전달합니다.
    retriever=retriever,
    llm=llm,
    )
    return multiquery_retriever
```

### ReRanker
```
reranker = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
compressor = CrossEncoderReranker(model=reranker, top_n=6)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)
```
### reOrder
```
def reorder_documents(docs):
    # 재정렬
    reordering = LongContextReorder()
    reordered_docs = reordering.transform_documents(docs)
    return reordered_docs
```



### chain
```
  chain = (
    {
        "context": itemgetter("question")
        | compression_retriever
        | RunnableLambda(reorder_documents),  # 질문을 기반으로 문맥을 검색합니다.
        "question": itemgetter("question"),  # 질문을 추출합니다.
    }
    | prompt  # 프롬프트 템플릿에 값을 전달합니다.
    | llm # 언어 모델에 프롬프트를 전달합니다.
    | StrOutputParser()  # 모델의 출력을 문자열로 파싱합니다.
    )

```

- context : 같은 데이터 증강
- 답변 품질 : 두 답변 모두 적절함
- perplexity 평가 : gpt 모델의 답변이 더 구체적이며, 설득력이 높다고 판단함

- 총평 : 성능 뿐아니라 속도 측면에서도 gpt 모델이 매우 우수함
